# Behavrioal Cloning Project (for debugging) #

This is a toy model, mainly used for DEBUGGING. Please visit model_rv.ipynb for the corrected version.

1. The [sample training data](https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip) is used for training model.

2. The model used navidia as a base, followed by one full connected layer. 

Links: 
* Simulators: [macOS](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/5831f290_simulator-macos/simulator-macos.zip), [Windows 64-bit](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/5831f3a4_simulator-windows-64/simulator-windows-64.zip), [Linux](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/5831f0f7_simulator-linux/simulator-linux.zip)



In [1]:
import csv
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

## Data

Data is collected as 'samples'. It has two columns: image directories and corresponding steering angles

In [2]:
samples = []

with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)
    for line in reader:
        image_dir = line[0]
        steering_angle = float(line[3])
        samples.append([image_dir,steering_angle])
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

## Data Preprocess and Generator

In [3]:
def crop_image(image):
    return image[60:140,:,:]

def generator(samples, batch_size=64):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './data/' + batch_sample[0]
                center_image = cv2.imread(name)
                center_image = crop_image(center_image)
                center_angle = batch_sample[1]
                images.append(center_image)
                angles.append(center_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

## Compile and train the model using the generator function

In [4]:
train_generator = generator(train_samples, batch_size=64)
validation_generator = generator(validation_samples, batch_size=64)

## Model Achitecture

Used Nvidia model as a base and followed by a full connected layer.

In [5]:
from keras.models import Sequential, Model
from keras.layers import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D
import tensorflow as tf

def resize_image(image, new_size=(66, 200)):
    return tf.image.resize_images(image, size=new_size, method=3, align_corners=False)

drop_prob = 0.5

model = Sequential()
# Preprocess incoming data, resize image to fit navidia architecture
model.add(Lambda(resize_image, input_shape=(80,320,3), output_shape=(66,200,3)))
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/127.5 - 1.))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Flatten())
#model.add(Dropout(drop_prob))
#model.add(Dense(512,activation='relu'))
#model.add(Dropout(drop_prob))
model.add(Dense(128,activation='relu'))
model.add(Dense(1))

Using TensorFlow backend.


## Train Model

In [6]:
model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch= len(train_samples), 
                    validation_data=validation_generator, nb_val_samples=len(validation_samples), 
                    nb_epoch=3)

Epoch 1/3
6428/6428 [==============================] - 95s - loss: 0.0110 - val_loss: 0.0094
Epoch 2/3
6428/6428 [==============================] - 88s - loss: 0.0094 - val_loss: 0.0087
Epoch 3/3
6428/6428 [==============================] - 89s - loss: 0.0091 - val_loss: 0.0085


## Save Model


model_json = model.to_json()
with open("model.json", "w", encoding="utf-8") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

